# Assignment 6 - EKF localization


## Submission guide

Submit the following:
1. Python code as a Jupyter notebook file.
3. A report inside the notebook summarizing the comparison results.
4. Create a folder with your name and roll number to upload the results.

### Submission deadline: End of the week (Friday)

---

# Extended Kalman Filter (EKF) Localization: Detailed Notes

## 1. **Introduction to Localization**
Localization is the process of determining a robot's position and orientation (pose) within a known or unknown environment. Accurate localization is critical for autonomous robots to navigate efficiently and safely. One of the most widely used techniques for this in robotics is the **Kalman Filter (KF)** and its nonlinear extension, the **Extended Kalman Filter (EKF)**.

## 2. **Why Use the Extended Kalman Filter (EKF)?**
The Kalman Filter (KF) is optimal for linear systems, but most robotic systems are nonlinear, which makes the basic KF unsuitable. The **Extended Kalman Filter** is used to handle **nonlinear** systems by linearizing them around the current estimate through a **first-order Taylor expansion**.

## 3. **EKF Localization Overview**
EKF localization provides a probabilistic approach to track the pose of a mobile robot over time by combining noisy sensor measurements and noisy control inputs. The robot’s state is described by a Gaussian probability distribution, and the EKF updates this distribution with each new measurement and movement command.

The process can be broken down into two main steps:
1. **Prediction Step (State Propagation)**: Predicts the robot's state based on motion models.
2. **Update Step (Measurement Correction)**: Updates the state estimate using sensor data (landmarks, laser scans, etc.).

## 4. **State Representation in EKF**
In 2D localization, the robot's pose can be represented as a vector:
\[
x = \begin{bmatrix}
x_r \\
y_r \\
\theta_r
\end{bmatrix}
\]
where:
- \( x_r \) and \( y_r \) are the robot's coordinates,
- \( \theta_r \) is the robot's orientation (yaw angle).

## 5. **Motion Model**
The motion model describes how the robot moves from one state to another. It is typically nonlinear and can be expressed as:

\[
x_{k+1} = f(x_k, u_k) + w_k
\]
where:
- \( x_{k+1} \) is the predicted state at the next time step,
- \( f(x_k, u_k) \) is the nonlinear motion model,
- \( u_k \) is the control input (velocity, angular velocity),
- \( w_k \) is the process noise (uncertainties in control inputs).

For example, if the control inputs are linear velocity \( v \) and angular velocity \( \omega \), the motion model can be written as:
\[
\begin{aligned}
x_{k+1} &= x_k + v_k \Delta t \cos(\theta_k) \\
y_{k+1} &= y_k + v_k \Delta t \sin(\theta_k) \\
\theta_{k+1} &= \theta_k + \omega_k \Delta t
\end{aligned}
\]
where \( \Delta t \) is the time interval between time steps.

## 6. **Measurement Model**
The measurement model relates the robot’s sensor readings to its position in the environment. It is also often nonlinear and can be expressed as:

\[
z_k = h(x_k) + v_k
\]
where:
- \( z_k \) is the measurement vector (e.g., range and bearing to landmarks),
- \( h(x_k) \) is the nonlinear measurement function,
- \( v_k \) is the measurement noise.

For example, if the robot is observing a landmark at position \( (x_l, y_l) \), the measurement might be:
\[
\begin{aligned}
r &= \sqrt{(x_l - x_r)^2 + (y_l - y_r)^2} \\
\phi &= \text{atan2}(y_l - y_r, x_l - x_r) - \theta_r
\end{aligned}
\]
where \( r \) is the range to the landmark, and \( \phi \) is the bearing.

# 7. **EKF Algorithm Steps**
The EKF consists of two phases: **Prediction** and **Correction**.

## 7.1 Prediction Step (Motion Update)
1. **State Prediction**: 
   \[
   \hat{x}_{k+1|k} = f(\hat{x}_k, u_k)
   \]
   This uses the control inputs and the motion model to predict the new state.

2. **Covariance Prediction**: 
   \[
   P_{k+1|k} = F_k P_k F_k^T + Q_k
   \]
   where:
   - \( F_k \) is the **Jacobian** of the motion model with respect to the state:
     \[
     F_k = \frac{\partial f(x_k, u_k)}{\partial x_k}
     \]
   - \( Q_k \) is the process noise covariance matrix.

# 7.2 Correction Step (Measurement Update)
1. **Innovation (Measurement Residual)**: The difference between predicted measurement and actual measurement:
   \[
   y_k = z_k - h(\hat{x}_{k|k-1})
   \]

2. **Innovation Covariance**: 
   \[
   S_k = H_k P_{k+1|k} H_k^T + R_k
   \]
   where:
   - \( H_k \) is the **Jacobian** of the measurement model with respect to the state:
     \[
     H_k = \frac{\partial h(x_k)}{\partial x_k}
     \]
   - \( R_k \) is the measurement noise covariance.

3. **Kalman Gain**: The Kalman gain determines how much weight to give to the new measurements:
   \[
   K_k = P_{k+1|k} H_k^T S_k^{-1}
   \]

4. **State Update**: The state estimate is updated using the Kalman gain and the measurement residual:
   \[
   \hat{x}_{k+1} = \hat{x}_{k+1|k} + K_k y_k
   \]

5. **Covariance Update**: The uncertainty in the state estimate is updated:
   \[
   P_{k+1} = (I - K_k H_k) P_{k+1|k}
   \]
   where \( I \) is the identity matrix.

## 8. **Jacobian Matrices**
EKF relies on linearizing nonlinear functions. The **Jacobian matrix** is used for this purpose, representing partial derivatives of the motion and measurement models.

For example, if the motion model is nonlinear, the **Jacobian of the motion model** (with respect to the state) is:
\[
F_k = \frac{\partial f(x_k, u_k)}{\partial x_k}
\]

Similarly, for a nonlinear measurement model, the **Jacobian of the measurement model** is:
\[
H_k = \frac{\partial h(x_k)}{\partial x_k}
\]

## 9. **Covariance Matrices**
- **Process Noise Covariance** (\( Q_k \)): Represents uncertainty in the motion model.
- **Measurement Noise Covariance** (\( R_k \)): Represents uncertainty in sensor measurements.

These matrices are often derived from sensor specifications or experimentally tuned.

## 10. **Advantages and Disadvantages of EKF**
- **Advantages**:
  - Efficient and fast due to linearization.
  - Works well for mildly nonlinear systems.
  - Widely used and well-studied in robotics.

- **Disadvantages**:
  - Linearization can introduce inaccuracies, especially in highly nonlinear systems.
  - Not robust to large errors or highly noisy environments (in those cases, other techniques like Particle Filters or Unscented Kalman Filters (UKF) might perform better).

## 11. **Applications of EKF Localization**
EKF is used in various real-world applications, including:
- Mobile robot navigation.
- GPS-based tracking.
- Self-driving cars.
- Augmented reality systems for tracking the user’s position.

## 12. **Conclusion**
EKF localization is a powerful tool that combines sensor measurements and motion models to estimate a robot’s pose. It handles nonlinearity by linearizing around the current estimate and updating the state with each movement and measurement. While highly effective, its accuracy depends on the linearization quality and sensor noise characteristics.